In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.ndimage import gaussian_filter
import tqdm
from joblib import Parallel, delayed
from markov_functions import *
from itertools import product
import warnings


In [15]:
pa_pitches_filename = './data/paired_filtered.csv'  # or provide a full path

if not os.path.exists(pa_pitches_filename):
    csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError(f"{pa_pitches_filename!r} not found and no CSV files in the current directory.")
    filename = csv_files[0]
    print(f"No file named 'data.csv' found. Using first CSV in cwd: {filename}")



pas = pd.read_csv(pa_pitches_filename)
pas.drop(['pa_seq', 'bases', 'velocities_events', 'pitchCodes_events', 'coords_events', 'n_pitches', 'ball_strike_counts', 'pitchTypes_events'], axis=1, inplace=True)
print(f"Loaded {len(pas)} rows and {len(pas.columns)} columns from {pa_pitches_filename}")
# pas.head()


Loaded 27600 rows and 9 columns from ./data/paired_filtered.csv


In [16]:
test = pas[(pas['hardness']=='H')][:50]

In [17]:
test_hit = test[test['result'].isin(['1B', '2B', '3B', 'HR', 'IHR', 'H'])]
test_hit.head(20)

,batterName,batterHand,pitcherName,pitcherHand,catcherName,pitchCodes,result,hardness,PAId
2,梁家榮,L,徐若熙,R,蔣少宏,"['S', 'F', 'B', 'H']",2B,H,1_2
3,廖健富,L,徐若熙,R,蔣少宏,"['B', 'H']",1B,H,1_3
19,林立,R,銳歐,L,蔣少宏,"['B', 'S', 'S', 'H']",1B,H,1_19
20,梁家榮,L,銳歐,L,蔣少宏,"['S', 'H']",1B,H,1_20
26,林智平,R,銳歐,L,蔣少宏,"['B', 'H']",2B,H,1_26
29,梁家榮,L,陳禹勳,R,蔣少宏,"['B', 'F', 'B', 'B', 'SW', 'H']",1B,H,1_29
37,李凱威,L,威能帝,R,嚴宏鈞,"['B', 'B', 'S', 'H']",2B,H,1_37
39,郭嚴文,L,威能帝,R,嚴宏鈞,"['B', 'F', 'H']",2B,H,1_39
41,蔣少宏,R,威能帝,R,嚴宏鈞,"['B', 'SW', 'H']",1B,H,1_41
42,陳品捷,L,威能帝,R,嚴宏鈞,"['F', 'B', 'B', 'B', 'H']",1B,H,1_42
